In [17]:
import pinecone
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [2]:
# Load environment variables
load_dotenv()

True

# Setting up Tools

In [3]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index("restaurant-info-medium-testing")

In [4]:
embeddings_model = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'),
                                        model="text-embedding-3-small")

In [5]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

In [6]:
search = TavilySearchResults(max_results=3)

In [7]:
search.invoke("Where is sushi in Los Angeles")

[{'url': 'https://www.yelp.com/search?cflt=sushi&find_loc=Los+Angeles,+CA',
  'content': 'Best Sushi Bars in Los Angeles, CA - Sushi by Scratch Restaurants: LA, Sushi Gen, Yeon Omakase and Sushi, Iki Nori, Hato Sushi, Izakaya Osen - Los Angeles, Sakana Sushi Lounge DTLA, Sushi Ye, Murakami, Ariyoshi Sushi Cafe Izakaya'},
 {'url': 'https://www.timeout.com/los-angeles/restaurants/best-sushi-restaurants-in-la',
  'content': "The best sushi in Los Angeles. Whether you're looking to splurge or save, the best L.A. sushi bars will satisfy your craving for immaculately cut raw fish. Tuesday August 27 2024."},
 {'url': 'https://www.yelp.com/search?find_desc=sushi&find_loc=Los+Angeles,+CA',
  'content': 'Top 10 Best sushi Near Los Angeles, California. 1. Yeon Omakase and Sushi. 2. Izakaya Osen - Los Angeles. 3. Chubby Cattle Little Tokyo. 4. Hato Sushi.'}]

In [8]:
retriever = vector_store.as_retriever(k=1)

In [9]:
retriever.invoke("where is sushi")[0]

Document(id='fc165d3d-c44c-470c-9217-b83cfe78a4cf', metadata={'address': '116 Japanese Village Plaza Mall, Los Angeles, CA 90012, USA', 'name': 'Sushi & Teri', 'place_id': 'ChIJizyr7UfGwoARt2VuQwQ0eIw', 'rating': 4.1}, page_content='Sushi & Teri is a small, inexpensive Japanese restaurant located in Little Tokyo, offering a variety of sushi rolls, ramen, miso soup, and cucumber salad. Customers generally feel that the food is decent but not exceptional, with notable mentions of the Alaska and Baja rolls, Washington roll, ramen served piping hot, and the cucumber salad. Key pros include friendly service, affordable prices, and the convenience of ordering via tablet, while cons revolve around long wait times, understaffing issues, and smaller portion sizes compared to nearby restaurants.')

In [10]:
retriever_tool = create_retriever_tool(
    retriever,
    "restaurant_search",
    "Search for information about restaurants in Los Angeles. If user prompts for restauraunts, USE THIS TOOL BY DEFAULT!",
)

In [11]:
tools = [search, retriever_tool]

In [12]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [13]:
llm.invoke("where is lebron currently", temperature=0.1)

AIMessage(content="As of October 2023, LeBron James is playing for the Los Angeles Lakers in the NBA. If you need more specific or updated information, please check the latest sports news or the NBA's official website.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 12, 'total_tokens': 55, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1806b6e-808e-428b-a17a-834054b090b7-0', usage_metadata={'input_tokens': 12, 'output_tokens': 43, 'total_tokens': 55})

In [14]:
prompt = hub.pull("hwchase17/openai-functions-agent")

## Creating Agent + Executor

In [15]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [18]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)